<img src="https://radar.community.uaf.edu/wp-content/uploads/sites/667/2023/08/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> EarthScope InSAR Training <b> </font>

<font size="5"> <b>Monday | Homework 1: Volcano Source Modeling Using InSAR  </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks </b> <br>

This lab will introduce you to the intersection between Radar Remote Sensing and Inverse Modeling. Radar Remote Sensing can provide you with geodetic observations of surface displacement. Inverse Modeling helps you understand the physical causes behind an observed displacement. 
    
To illuminate the handoff from geodesy to geophysics, this lab will show how to use InSAR observations to determine the most likely parameters of a volcanic magma source underneath Okmok volcano, Alaska. You will use a Mogi source model to describe the physics behind displacement at Okmok. <br><br>

<b>This Lab will introduce the following data analysis concepts:</b>
- A Mogi Source Model describing volcanic source geometry and physics
- How to use the "grid search" method to perform a pseudo-inversion of a Mogi source model 
- How to solve for the best fitting source parameters using modeling with InSAR data
</font>

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/earthscope_insar':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "earthscope_insar" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "earthscope_insar" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "earthscope_insar" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

<div class="alert alert-danger">
<font face="Calibri" size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES A HOMEWORK ASSIGNMENT (Section 5)!</b></font> 
<br>
The homework assignment in this lab is to be completed within this notebook. To submit your homework, please use the following process:

<ol type="1">
  <li><font color='rgba(200,0,0,0.2)'> <b> Export your completed notebook as html</b></font> using the <i>file / Save and Export Notebook as / Html embedded</i> feature in the Jupyter Hub menu
  <li><font color='rgba(200,0,0,0.2)'> <b>Submit your homework to our Course Homework space</b></font>: To do so, navigate to our <a href="https://drive.google.com/drive/folders/1mVOyu7jMny55PwL-PS7l5XhT21GH14Gd" target="_blank">Homework Space</a>. Follow the homework submission instructions located <a href="https://docs.google.com/document/d/1yiOOd_RlodSpaPoVtbgyMmJRtmLofPPrDz_0W2OIyIM/edit?usp=sharing" target="_blank">here</a>.
</ol>    

Contact me at fjmeyer@alaska.edu or via Slack should you run into any problems.
</font>
</font>
</div>

<hr>

# 0. Importing Relevant Python Packages

First step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following scientific libraries:
<ol type="1">
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. </li>
</ol>
<br>
The first step is to <b>import all required python modules:</b>

In [ ]:
import os # for chdir, getcwd, path.basename, path.exists
import copy
import subprocess # for check_call

import matplotlib.pylab as plt # for add_subplot, cm.jet, colorbar, figure, grid, imshow, rcParams.update, savefig,
                               # set_bad, set_clim, set_title, set_xlabel, set_ylabel
import numpy as np # for arange, arctan, concatenate, cos, fromfile, isnan, ma.masked_value, min, pi, power, reshape,
                   # sqrt, square, sin, sum, tile, transpose, where, zeros     

<hr>

# 1. Introduction to the Study Site: Okmok Volcano, Alaska

<img style="padding: 7px" src="https://courses.edx.org/asset-v1:AlaskaX+SAR-401+3T2020+type@asset+block@Lab6-OkmokdefoGPS.JPG" width="550" align="right" /> Okmok is one of the more active volcanoes in Alaska’s Aleutian Chain. Its last (confirmed) eruption was in the summer of 2008. Okmok is interesting from an InSAR perspective as it inflates and deflates heavily as magma moves around in its magmatic source located roughly 2.5 km underneath the surface. To learn more about Okmok volcano and its eruptive history, please visit the very informative site of the <a href="https://avo.alaska.edu/volcanoes/activity.php?volcname=Okmok&eruptionid=604&page=basic" target="_blank">Alaska Volcano Observatory</a>.

This lab uses a pair of C-band ERS-2 SAR images acquired on Aug 18, 2000 and Jul 19, 2002 to analyze the properties of a volcanic source that was responsible for an inflation of Okmok volcano of more than 3 cm near its summit. The figure to the right shows the Okmok surface displacement as measured by GPS data from field campaigns conducted in 2000 and 2002. The plots show that the displacement measured at the site is consistent with that created by an inflating point (Mogi) source.<br> 

<b>The primary goal of the problem set is to estimate values for four unknown model parameters describing a source process beneath a volcano.</b> The lab uses real InSAR data from Okmok volcano, so you should get some sense for how remote sensing can be used to infer physical processes at volcanoes. We will assume that the source can be modeled as an inflating point source (a so-called Mogi source) and will use a grid-search method to find the source model parameters (3D source location and volume of magma influx) that best describe our InSAR-observed surface displacement.
<hr>

# 2. Downloading and Visualizing the InSAR Data

## 2.1 Download Data from AWS S3 Storage Bucket and Prep for Further Processing

We are using a pre-calculated displacement map created from C-band ERS-2 SAR images acquired on Aug 18, 2000 and Jul 19, 2002. We will pull the displacement map from an <b>Amazon Web Services (AWS) S3 storage bucket:</b>

<b>Create and move to a directory in which to store our Lab 2 files</b>:

In [ ]:
path = f"{os.getcwd()}/lab_6_data"
if not os.path.exists(path):
    os.makedirs(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<b>Download the displacement map from the AWS-S3 bucket:</b>

In [ ]:
displacement_map_path = 's3://asf-jupyter-data-west/E451_20000818_20020719.unw'
displacement_map = os.path.basename(displacement_map_path)
!aws --region=us-west-2 --no-sign-request s3 cp $displacement_map_path $displacement_map

<b>Define some variables:</b>

In [ ]:
sample = 1100
line = 980
posting = 40.0
half_wave = 28.3

<b>Read the dataset into the notebook</b>, storing our InSAR data in the variable <i>"observed_displacement_map"</i>: 

In [ ]:
if os.path.exists(displacement_map):
    with open (displacement_map, 'rb') as f:    
        coh = np.fromfile(f, dtype='>f', count=-1)
    observed_displacement_map = np.reshape(coh, (line, sample))

Now we scale the measured and unwrapped InSAR phase into surface displacement in *cm* units and <b>replace all ```nans``` with 0</b>

In [ ]:
observed_displacement_map = observed_displacement_map*half_wave/2.0/np.pi
where_are_NaNs = np.isnan(observed_displacement_map)
observed_displacement_map[where_are_NaNs] = 0

 <b>Create a mask</b> that removes invalid samples (low coherence) from the displacement map:

In [ ]:
observed_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, observed_displacement_map)

<hr>

## 2.2 Visualize The Surface Displacement Map

We will visualize the displacement map both in units of [cm] and as a rewrapped interferogram.
<br><br>
<b>Write a function that calculates the bounding box.</b>

In [ ]:
def extents(vector_component):
        delta = vector_component[1] - vector_component[0]
        return [vector_component[0] - delta/2, vector_component[-1] + delta/2]

<b>Create a directory in which to store the plots we are about to make, and move into it:</b>

In [ ]:
os.chdir(path)
product_path = 'plots'
if not os.path.exists(product_path):
    os.makedirs(product_path)
if os.path.exists(product_path) and os.getcwd() != f"{path}/{product_path}":
    os.chdir(product_path)
print(f"Current working directory: {os.getcwd()}")

<b>Write a plotting function</b>:

In [ ]:
def plot_model(infile, line, sample, posting, output_filename=None, dpi=72):
    # Calculate the bounding box
    extent_xvec = extents((np.arange(1, sample*posting, posting)) / 1000)
    extent_yvec = extents((np.arange(1, line*posting, posting)) / 1000)
    extent_xy = extent_xvec + extent_yvec
    
    plt.rcParams.update({'font.size': 14})
    inwrapped = (infile/10 + np.pi) % (2*np.pi) - np.pi
    cmap = copy.copy(plt.colormaps["jet"])
    cmap.set_bad('white', 1.)
    
    # Plot displacement
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(1, 2, 1)
    im = ax1.imshow(infile, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax1.figure.colorbar(im, ax=ax1, orientation='horizontal')
    ax1.set_title("Displacement in look direction [mm]")
    ax1.set_xlabel("Easting [km]")
    ax1.set_ylabel("Northing [km]")
    plt.grid()
    
    # Plot interferogram
    im.set_clim(-30, 30)
    ax2 = fig.add_subplot(1, 2, 2)
    im = ax2.imshow(inwrapped, interpolation='nearest', cmap=cmap, extent=extent_xy, origin='upper')
    cbar = ax2.figure.colorbar(im, ax=ax2, orientation='horizontal')
    ax2.set_title("Interferogram phase [rad]")
    ax2.set_xlabel("Easting [km]")
    ax2.set_ylabel("Northing [km]")
    plt.grid()
    
    if output_filename:
        plt.savefig(output_filename, dpi=dpi)

Call plot_model() to <b>plot our observed displacement map:</b>

In [ ]:
plot_model(observed_displacement_map_m, line, sample, posting, output_filename='Okmok-inflation-observation.png', dpi=200)

<hr>

# 3. The Mogi Source Forward Model for InSAR Observations

## 3.1 The Mogi Equation

The Mogi model provides the 3D ground displacement, $u(x,y,z)$, due to an inflating source at location $(x_s,y_s,z_s)$ with volume change $V$:

\begin{equation}
u(x,y,z)=\frac{1}{\pi}(1-\nu)\cdot V\Big(\frac{x-x_s}{r(x,y,z)^3},\frac{y-y_s}{r(x,y,z)^3},\frac{z-z_s}{r(x,y,z)^3}\Big)
\end{equation}
<br>
\begin{equation}
r(x,y,z)=\sqrt{(x-x_s)^2+(y-y_s)^2+(z-z_s)^2}
\end{equation}

where $r$ is the distance from the Mogi source to $(x,y,z)$, and $\nu$ is the Poisson's ratio of the halfspace. The Poisson ratio describes how rocks react when put under stress (e.g., pressure). It is affected by temperature, the quantity of liquid to solid, and the composition of the soil material. <b>In our problem, we will assume that $\nu$ is fixed</b>. 

## 3.2 Projecting Mogi Displacement to InSAR Line-of-Sight

In our example, the $x$-axis points east, $y$ points north, and $z$ points up. However, in the code the input values for $z$ are assumed to be depth, such that the Mogi source is at depth $z_s > 0$. The observed interferogram is already corrected for the effect of topography, so the observations can be considered to be at $z = 0$.
    
<img style="padding: 7px" src="https://courses.edx.org/asset-v1:AlaskaX+SAR-401+3T2020+type@asset+block@Lab6-LOSprojection.JPG" width="650" align="center" />
The satellite “sees” a projection of the 3D ground displacement, $u$, onto the look vector, $\hat{L}$, which points from the satellite to the target. Therefore, we are actually interested in the (signed magnitude of the) projection of $u$ onto $\hat{L}$ (right). This is given by

\begin{array}{lcl} proj_{\hat{L}}u & = & (u^T\hat{L})\hat{L} \\ u^T\hat{L} & = & u \cdot \hat{L} = |u||\hat{L}|cos(\alpha) = |u|cos(\alpha) \\ & = & u_x\hat{L}_x+ u_y\hat{L}_y + u_z\hat{L}_z \end{array}

where the look vector is given by $\hat{L}=(sin(l) \cdot cos(t), -sin(l) \cdot sin(t), -cos(l))$, where $l$ is the look angle measured from the nadir direction and $t$ is the satellite track angle measured clockwise from geographic north. All vectors are represented in an east-north-up basis.

Our forward model takes a Mogi source, $(x_s,y_s,z_s,V)$, and computes the look displacement at any given $(x, y, z)$ point. If we represent the <i>i</i>th point on our surface grid by $x_i = (x_i,y_i,z_i)$ the the displacement vector is $u_i = u(x_i, y_i, z_i)$, and the look displacement is

\begin{equation}
d_i = u_i \cdot \hat{L}
\end{equation}



## 3.3 Defining the Mogi Forward Model

We can now represent the Mogi <i>forward problem</i> as 

\begin{equation}
g(m) = d
\end{equation}

where $g(·)$ describes the forward model in the very first equation in this notebook, $m$ is the (unknown) Mogi model, and $d$ is the predicted interferogram. The following code cells calculate the Mogi forward model according to the equations given above:

<font face="Calibri" size="3"><b>Write a function to calculate a forward model for a Mogi source.</b> </font>

In [ ]:
def calc_forward_model_mogi(n1, e1, depth, delta_volume, northing, easting, plook):
    
    # This geophysical coefficient is needed to describe how pressure relates to volume change
    displacement_coefficient = (1e6*delta_volume*3)/(np.pi*4)
    
    # Calculating the horizontal distance from every point in the displacement map to the x/y source location
    d_mat = np.sqrt(np.square(northing-n1) + np.square(easting-e1))
    
    # denominator of displacement field for mogi source
    tmp_hyp = np.power(np.square(d_mat) + np.square(depth),1.5)
    
    # horizontal displacement
    horizontal_displacement = displacement_coefficient * d_mat / tmp_hyp
    
    # vertical displacement
    vertical_displacement = displacement_coefficient * depth / tmp_hyp
    
    # azimuthal angle
    azimuth = np.arctan2((easting-e1), (northing-n1))
    
    # compute north and east displacement from horizontal displacement and azimuth angle
    east_displacement = np.sin(azimuth) * horizontal_displacement
    north_displacement = np.cos(azimuth) * horizontal_displacement
    
    # project displacement field onto look vector
    temp = np.concatenate((east_displacement, north_displacement, vertical_displacement), axis=1)
    delta_range = temp.dot(np.transpose([plook]))
    delta_range = -1.0 * delta_range
    return delta_range

<b>Write a function to create simulated displacement data based on Mogi Source Model parameters:</b> 

In [ ]:
def displacement_data_from_mogi(x, y, z, volume, iplot, imask):
    # Organizing model parameters
    bvc = [x, y, z, volume, 0, 0, 0, 0]
    bvc = np.asarray(bvc, dtype=object)
    bvc = np.transpose(bvc)
    
    # Setting acquisition parameters
    track =  -13.3*np.pi / 180.0
    look  = 23.0*np.pi / 180.0
    plook = [-np.sin(look)*np.cos(track), np.sin(look)*np.sin(track), np.cos(look)]
    
    # Defining easting and northing vectors
    northing = np.arange(0, (line)*posting, posting) / 1000
    easting = np.arange(0, (sample)*posting, posting) / 1000
    northing_mat = np.tile(northing, (sample, 1))
    easting_mat = np.transpose(np.tile(easting, (line, 1)))
    northing_vec = np.reshape(northing_mat, (line*sample, 1))
    easting_vec = np.reshape(easting_mat, (line*sample, 1))
    
    # Handing coordinates and model parameters over to the rngchg_mogi function
    calc_range = calc_forward_model_mogi(bvc[1], bvc[0], bvc[2], bvc[3], northing_vec, easting_vec, plook)
    
    # Reshaping surface displacement data derived via calc_forward_model_mogi()
    surface_displacement = np.reshape(calc_range, (sample,line))
    
    # return rotated surface displacement
    return np.transpose(np.fliplr(surface_displacement))

<hr>

## 3.4 Plotting The Mogi Forward Model

The cell below plots several Mogi forward models by varying some of the four main Mogi modeling parameters $(x_s,y_s,z_s,V)$.
    
The examples below fix the <i>depth</i> parameter to $z_s = 2.58 km$ and the <i>volume</i> change parameter to $volume = 0.0034 km^3$. We then vary the <i>easting</i> and <i>northing</i> parameters $x_s$ and $y_s$ to demonstrate how the model predictions vary when model parameters are changed.
<br><br>
<b>Run the first example:</b> 

In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, sample*posting, posting))/1000)
extent_y = extents((np.arange(1, line*posting, posting))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

zs = 2.58;
volume = 0.0034;
xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = copy.copy(plt.colormaps["jet"])
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_displacement_map = displacement_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, predicted_displacement_map)
        im = ax.imshow(predicted_displacement_map_m, interpolation='nearest', cmap=cmap, extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k],ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title('Source: X=%4.2fkm; Y=%4.2fkm' % (xs[k], ys[l]))
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1
        
plt.savefig('Model-samples-3by3.png', dpi=200, transparent='false')

<br>
<hr>
<div class="alert alert-warning">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>SUGGESTED EXERCISE</u>:  </font> Experiment with the Mogi Forward Model </b>  </font>

To get a feeling for the Mogi forward model, please run the following forward model experiments using the Python Function <i>displacement_data_from_mogi</i> and plot the results (using the code cell above):

<ol type="a">
<li>Run a reference simulation using the code cell above by specifying the following model parameters for source depth $z_s$ and volume change $V$: $z_{s1} = 2.5 km$; $V_1 = 0.01 km^3$. The script will visualize the resulting simulated surface displacement maps. Change the name of the output figure (last line of the script) to something that you will recognize later on (e.g., ReferenceRun.png).</li>
<br>
<li>Change the depth of the source by a factor of three ($z_{s2} = 7.5 km$) while leaving the other model parameters unchanged. Modify name of the output figure in the last line of the script. Visualize the results. Discuss changes to the reference run. Describe how the strength and shape of the displacement signal has changed and provide a physical explanation.</li>
<br>
<li>Now change the source volume by a factor of three ($V_2 = 0.03 km^3$ – also reset source depth to $z_{s1} = 2.5 km$). Visualize the results and compare them to the reference run.</li>
<br>
<li>Finally change both source volume and depth by a factor of three ($z_{s2} = 7.5 km$ and $V_2 = 0.03 km^3$). Compare this result to the results of experiments 1–3.</li>
</ol>
</div>

<hr>
<b>You can modify the example script below to work on exercises 1 - 4 above:</b>

In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, sample*posting, posting))/1000)
extent_y = extents((np.arange(1, line*posting, posting))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

# ------------ Change Variables HERE --------------- #
zs = 2.58;
volume = 0.0034;
# ------------------------------------------------- #

xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = copy.copy(plt.cm.get_cmap("jet"))
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_displacement_map = displacement_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, predicted_displacement_map)
        im = ax.imshow(predicted_displacement_map_m, interpolation='nearest', cmap=cmap,extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation ='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k],ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title(f"Source: X={xs[k]:.2f}km; Y={ys[l]:.2f}km")
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1

# CHANGE THE NAME OF THE IMAGE THAT IS BEING SAVED TO YOUR CLOUD INSTANCE!!
plt.savefig('Model-samples-3by3.png', dpi=200, transparent='false')

<hr>

# 4. Solving the Inverse Model

The inverse problem seeks to determine the optimal parameters $(\hat{x_s},\hat{y_s},\hat{z_s},\hat{V})$ of the Mogi model $m$ by minimizing the <i>misfit</i> between predictions, $g(m)$, and observations $d^{obs}$ according to
    
\begin{equation}
\sum{\Big[g(m) - d^{obs}\Big]^2}
\end{equation}

This equation describes misfit using the <i>method of least-squares</i>, a standard approach to approximate the solution of an over-determined equation system. We will use a <i>grid-search</i> approach to find the set of model parameters that minimize the the misfit function. The approach is composed of the following processing steps: 
<ol>
<li>Loop through the Mogi model parameters,</li> 
<li>Calculate the forward model for each set of parameters,</li> 
<li>Calculate the misfit $\sum{[g(m) - d^{obs}]^2}$, and</li> 
<li>Find the parameter set that minimizes this misfit.</li>
</ol>

## 4.1 Experimenting with Misfit

Let's <b>look at the misfit $\sum{[g(m) - d^{obs}]^2}$ for a number of different model parameter sets $(x_s,y_s,z_s,V)$:</b> 


In [ ]:
plt.rcParams.update({'font.size': 14})
extent_x = extents((np.arange(1, sample*posting, posting))/1000)
extent_y = extents((np.arange(1, line*posting, posting))/1000)
extent_xy = extent_x + extent_y
xs = np.arange(18, 24.2, 0.4)
ys = np.arange(20, 24.2, 0.4)

# ------------ Change Variables HERE --------------- #
zs = 2.58;
volume = 0.0034;
# ------------------------------------------------- #

xa = [0, 7, 15]
ya = [0 ,5, 10]

fig = plt.figure(figsize=(18, 18))
cmap = copy.copy(plt.colormaps["jet"])
subplot_index = 1

for k in xa:
    for l in ya: 
        ax = fig.add_subplot(3, 3, subplot_index)
        predicted_displacement_map = displacement_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, predicted_displacement_map)
        im = ax.imshow(predicted_displacement_map_m, interpolation='nearest', cmap=cmap,extent=extent_xy)
        cbar = ax.figure.colorbar(im, ax=ax, orientation ='horizontal')
        plt.grid()
        im.set_clim(-30, 30)
        ax.plot(xs[k],ys[l], 'k*', markersize=25, markerfacecolor='w')
        ax.set_title(f"Source: X={xs[k]:.2f}km; Y={ys[l]:.2f}km")
        ax.set_xlabel("Easting [km]")
        ax.set_ylabel("Northing [km]")
        subplot_index += 1

# CHANGE THE NAME OF THE IMAGE THAT IS BEING SAVED TO YOUR CLOUD INSTANCE!!
plt.savefig('Model-samples-3by3.png', dpi=200, transparent='false')

<hr>

## 4.2 Running Grid-Search to find Best Fitting Model Parameter $(\hat{x}_s,\hat{y}_s)$

The following code cell runs a grid-search approach to find the best fitting Mogi source parameters for the 2000-2002 displacement event at Okmok. To keep things simple, we will fix the depth $z_s$ and volume change $V$ parameters close to their "true" values and search only for the correct east/north source location ($x_s,y_s$).</font>
<br><br>
<b>Write a script using the grid-search approach in Python:</b>

In [ ]:
# FIX Z AND dV, SEARCH OVER X AND Y

# Setting up search parameters
xs = np.arange(19, 22.2, 0.2)
ys = np.arange(21, 23.2, 0.2)
zs = 2.58;
volume = 0.0034;

nx = xs.size
ny = ys.size
ng = nx * ny;

print(f"fixed z = {zs}km, dV = {volume}, searching over (x,y)")

misfit = np.zeros((nx, ny))
subplot_index = 0

# Commence grid-search for best model parameters
for k, xv in enumerate(xs):
    for l, yv in enumerate(ys):
        subplot_index += 1
        predicted_displacement_map = displacement_data_from_mogi(xs[k], ys[l], zs, volume, 0, 0)
        predicted_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, predicted_displacement_map)
        misfit[k,l] = np.sum(np.square(observed_displacement_map_m - predicted_displacement_map_m))
    print(f"Source {subplot_index:3d}/{ng:3d} is x = {xs[k]:.2f} km, y = {ys[l]:.2f} km")

# Searching for the minimum in the misfit matrix
mmf = np.where(misfit == np.min(misfit))
print(f"\n----------------------------------------------------------------")
print('Best fitting Mogi Source located at: X = %5.2f km; Y = %5.2f km' % (xs[mmf[0]], ys[mmf[1]]))
print(f"----------------------------------------------------------------")

<hr>

## 4.3 Plot and Inspect the Misfit Function

The code cell below plots the misfit function ($\sum{[g(m) - d^{obs}]^2}$) describing the fit of different Mogi source parameterizations to the observed InSAR data. You should notice a clear minimum in the misfit plot at the location of the best fitting source location estimated above (the best fitting model is indicated by a star in the plot). 
    
You may notice that, even for the best fitting solution, the misfit does not become zero. This could be due to other signals in the InSAR data (e.g., atmospheric effects or residual topography). Alternatively, it could also indicate that the observed displacement doesn't fully match what is predicted by the Mogi theory. 
</font>
<br><br>
<b>Plot the misfit function ($\sum{[g(m) - d^{obs}]^2}$):</b>

In [ ]:
plt.rcParams.update({'font.size': 18})
extent_xy = extents(xs) + extents(ys)
fig = plt.figure(figsize=(10, 10))
cmap = copy.copy(plt.colormaps["jet"])
ax1 = fig.add_subplot(1, 1 ,1)
im = ax1.imshow(np.transpose(misfit), origin='lower', interpolation='nearest', cmap=cmap, extent=extent_xy)
# USE THIS COMMAND TO CHANGE COLOR SCALING: im.set_clim(-30, 30)
ax1.set_aspect('auto')
cbar = ax1.figure.colorbar(im, ax=ax1, orientation='horizontal')
ax1.plot(xs[mmf[0]], ys[mmf[1]], 'k*', markersize=25, markerfacecolor='w')
ax1.set_title("Misfit Function for Mogi-Source Approximation")
ax1.set_xlabel("Easting [km]")
ax1.set_ylabel("Northing [km]")
plt.savefig('Misfit-function.png', dpi=200, transparent='false')

<hr>

## 4.4 Plot Best-Fitting Mogi Forward Model and Compare to Observations

With the best-fitting model parameters defined, you can now analyze how well the model fits the InSAR-observed surface displacement. The best way to do that is to look at both the observed and predicted displacement maps and compare their spatial patterns. Additionally, we will also plot the residuals (<i>observed_displacement_map</i> - <i>observed_displacement_map</i>) to determine if there are additional signals in the data that are not modeled using Mogi theory. 

### 4.4.1 Compare the Observed and Predicted Displacement Maps

In [ ]:
# Calculate predicted displacement map for best-fitting Mogi parameters:
predicted_displacement_map = displacement_data_from_mogi(xs[mmf[0]], ys[mmf[1]], zs, volume, 0, 0)

# Mask the predicted displacement map to remove pixels incoherent in the observations:
predicted_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, predicted_displacement_map)

# Plot observed displacement map
plot_model(observed_displacement_map_m, line, sample, posting)

# Plot simulated displacement map
plot_model(predicted_displacement_map_m, line, sample, posting)

plt.savefig('BestFittingMogiDefo.png', dpi=200, transparent='false')

# Plot simulated displacement map without mask applied
plot_model(predicted_displacement_map, line, sample, posting)

### 4.4.2 Plot Misfit to Determine if there are Additional Signals in the Data that are not Modeled using Mogi

We plot the difference between the observations and the best fitting model to assess whether the InSAR data contains other signals that are not modeled by Mogi.

In [ ]:
# Plot residual between observed and predicted displacement maps
plot_model(observed_displacement_map_m-predicted_displacement_map_m, line, sample, posting)
plt.savefig('Residuals-ObsMinusMogi.png', dpi=200, transparent='false')

<br>
<hr>
<div class="alert alert-warning">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>SUGGESTED EXERCISE</u>:  </font> Think About Why the Residuals are not Zero </b>  </font>

In a perfect world where data are noise-free and geophysical models perfectly represent reality, there should be a set of model parameters that reduces the misfit function to zero. In our case, however, the misfit function still shows large values, even for the best fitting model parameters. Think about why the differences between the model and the data might not be zero.
<br><br>
Differences may potentially arise from limitations of the model, potential InSAR signals unrelated to the volcanic event (the image below may remind you of one data artifact that could cause errors), or to processing errors, to name a few.

<img style="padding: 7px" src="https://courses.edx.org/asset-v1:AlaskaX+SAR-401+3T2022+type@asset+block@stratifieddelayexample.jpg" width="80%" align="center" />
</div>

<hr>

# <font color='rgba(200,0,0,0.2)'> 5. Homework Assignment</font> 

<br>
<div class="alert alert-danger">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #1</u>:  </font> Run 2nd Grid-Search to Find Model Parameters $(\hat{z}_s,\hat{V})$  </b> </font>

For this second grid-search run, we now switch out the model parameters we are trying to estimate. We will assume that the lateral location of the Mogi source is now fixed to its estimated value ($\hat{x}_s = 20.6 km$; $\hat{y}_s = 21.8 km$). 

<u>To perform a grid search for the best fitting model parameters $\hat{z}_s$ and $\hat{V}$, please complete the following steps</u>:

<ol>
<br>
<li>Using the previous grid-search script as a template, <b>write a new grid-search script to search for the best fitting source model depth ($z_s$) and volume change ($V$)</b>.</li>
<br>
<li>Provide a plot of the misfit function and provide the best-fitting values for $\hat{z}_s$ and $\hat{V}$. When plotting the misfit function, put $z_s$ on the vertical axis. You may want to adjust the color scale, in order to better see the shape of the misfit function.</li>
</ol>
</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 1.1:</font> Space for you to provide code to perform grid search over $z_s$ and $V$.</i> 

<b>PROVIDE SCRIPT BY MODIFYING THE CODE IN THE CODE CELL BELOW:</b>
</font>
</div>

In [ ]:
# !!!! MODIFY THIS SCRIPT TO PERFORM A GRID SEARCH OVER zs AND V: !!!!

# Setting up search parameters
xs = np.arange(19, 22.2, 0.2)
ys = np.arange(21, 23.2, 0.2)
zs = 2.58;
volume = 0.0034;

nx = xs.size
ny = ys.size
ng = nx * ny;

#print('fixed z = ',zs,' km, dV = ',volume, ' searching over (x,y)')
print(f"fixed z = {zs}km, dV = {volume}, searching over (x,y)")

misfit=np.zeros((nx,ny))
subplot_index = 0

# Commence grid-search for best model parameters
for k, xv in enumerate(xs):
    for l, yv in enumerate(ys):
        subplot_index = subplot_index+1
        predicted_displacement_map = displacement_data_from_mogi(xs[k],ys[l],zs,volume,0,0)
        predicted_displacement_map_m = np.ma.masked_where(observed_displacement_map == 0, predicted_displacement_map)
        misfit[k,l] = np.sum(np.square(observed_displacement_map_m - predicted_displacement_map_m))
    print(f"Source {subplot_index:3d}/{ng:3d} is x = {xs[k]:.2f} km, y = {ys[l]:.2f} km")

# Searching for the minimum in the misfit matrix
mmf = np.where(misfit == np.min(misfit))
print('')
print(f"\n----------------------------------------------------------------")
print('Best fitting Mogi Source located at: X = %5.2f km; Y = %5.2f km' % (xs[mmf[0]], ys[mmf[1]]))
print(f"----------------------------------------------------------------")

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 1.2-A:</font> Provide the best fitting values for source depth ($\hat{z}_s$) and volume change ($\hat{V}$) according to your grid-search results.</i> 

PROVIDE ESTIMATES FOR $\hat{x}_s$ and $\hat{V}$ HERE:
</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 1.2-B:</font> Provide plot of $z_s$ on $V$ misfit function.</i> 

PROVIDE PLOT HERE:
</div>

# 6. Version Log

<i>EarthScopeInSAR-GeophysicalModeling-HW2.ipynb - Version 1.4.3 - Aug 2023
    <br><br>
        <b>Version Changes:</b>
    <ul>
        <li>code updates to accommodate python changes</li>
        <li>updated code, removed asf_notebook references and updated text</li>
        <li>namespace asf_notebook</li>
        <li>Adjusted for unavco environment</li>
        <li>specified nearest-neighbor interpolation to fix odd plotting results when plotting masked arrays</li>
        <li>Some updates to paths</li>
        <li>Some updates to homework instructions and bug squashing</li>
    </ul>
    </i>
</font>